In [1]:
import pandas as pd
import numpy as np
from typing import Dict
import optuna

# --- All Project Imports ---
from AlgoTrade.Utils.DataManager import DataManager
from AlgoTrade.Config.BacktestConfig import BacktestConfig
from AlgoTrade.Config.Enums import TradingMode
from AlgoTrade.Strat.Class.BaseStrategy import BaseStrategy
from AlgoTrade.Factories.IndicatorFactory import IndicatorFactory
from AlgoTrade.Sizing.AtrBandsSizer import AtrBandsSizer
from AlgoTrade.Optimizer.StudyRunner import run_optimization
from AlgoTrade.Analysis.BacktestAnalysis import BacktestAnalysis

In [2]:
symbol_ltf = "TRX/USDT:USDT"
tframe_ltf = "1m"
symbol_htf = "TRX/USDT:USDT"
tframe_htf = "1h"
start_date = "2024-01-01"
dm = DataManager(name="bitget")

In [3]:
ltf_data = dm.from_local(symbol_ltf, tframe_ltf, start_date)
htf_data = dm.from_local(symbol_htf, tframe_htf, start_date)

Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\TRX_USDT_USDT_1m.csv...
Load completed successfully. Data shape: (773918, 5)
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\TRX_USDT_USDT_1h.csv...
Load completed successfully. Data shape: (12756, 5)


In [4]:
htf_data_resampled = htf_data.reindex(ltf_data.index, method="ffill")
merged = ltf_data.join(htf_data_resampled, rsuffix="_htf").dropna()

In [5]:
merged_analysis = merged[["close", "close_htf"]].copy()

In [6]:
merged_analysis["direction"] = np.where(merged_analysis["close"] > merged_analysis["close"].shift(1), "up", "down")

In [7]:
merged_analysis.iloc[:60]["direction"].value_counts()

direction
down    30
up      30
Name: count, dtype: int64

In [8]:
# Map direction to numeric values
direction_numeric = merged_analysis["direction"].map({"up": 1, "down": -1})

# One-hot style flags for up/down
is_up = (merged_analysis["direction"] == "up").astype(int)
is_down = (merged_analysis["direction"] == "down").astype(int)

# Rolling count of ups and downs
up_count = is_up.rolling(window=60, min_periods=1).sum()
down_count = is_down.rolling(window=60, min_periods=1).sum()

# Assign signal based on counts
merged_analysis["signal"] = np.select(
    [up_count > 30, down_count > 30], [1, -1], default=0
)

In [9]:
new_merged_analysis = merged_analysis.iloc[range(0, len(merged_analysis), 60)]

In [10]:
new_merged_analysis

,close,close_htf,direction,signal
date,,,,
2024-01-01 00:00:00,0.10779,0.10839,down,0
2024-01-01 01:00:00,0.10839,0.10822,down,0
2024-01-01 02:00:00,0.10829,0.10784,up,-1
2024-01-01 03:00:00,0.10784,0.10714,down,-1
2024-01-01 04:00:00,0.10714,0.10657,down,-1
...,...,...,...,...
2025-06-21 06:00:00,0.27327,0.27212,up,1
2025-06-21 07:00:00,0.27418,0.27212,down,-1
2025-06-21 08:00:00,0.27401,0.27212,up,-1
